# JHU CSSE GIS Dashboard
* https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6
* https://coronavirus.jhu.edu/map.html
* https://www.arcgis.com/home/item.html?id=bbb2e4f589ba40d692fab712ae37b9ac#data

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
data = requests.get(
    'https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&resultOffset=0&resultRecordCount=500&cacheHint=true&quantizationParameters=%7B%22mode%22%3A%22edit%22%7D').json()

In [3]:
country_list = []
for country in data['features']:
    country_dic = {}
    country_dic['OBJECTID'] = country['attributes']['OBJECTID']
    country_dic['Province_State'] = country['attributes']['Province_State']
    country_dic['Country_Region'] = country['attributes']['Country_Region']
    country_dic['Last_Update'] = country['attributes']['Last_Update']
    country_dic['Lat'] = country['attributes']['Lat']
    country_dic['Long_'] = country['attributes']['Long_']
    country_dic['Confirmed'] = country['attributes']['Confirmed']
    country_dic['Recovered'] = country['attributes']['Recovered']
    country_dic['Deaths'] = country['attributes']['Deaths']
    country_list.append(country_dic)

In [4]:
# data = requests.get(
#     'https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/2/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&resultOffset=0&resultRecordCount=200&cacheHint=true&quantizationParameters=%7B%22mode%22%3A%22edit%22%7D').json()

In [5]:
# country_list = []
# for country in data['features']:
#     country_dic = {}
#     country_dic['OBJECTID'] = country['attributes']['OBJECTID']
#     country_dic['Country_Region'] = country['attributes']['Country_Region']
#     country_dic['Last_Update'] = country['attributes']['Last_Update']
#     country_dic['Lat'] = country['attributes']['Lat']
#     country_dic['Long_'] = country['attributes']['Long_']
#     country_dic['Confirmed'] = country['attributes']['Confirmed']
#     country_dic['Deaths'] = country['attributes']['Deaths']
#     country_dic['Recovered'] = country['attributes']['Recovered']
#     country_list.append(country_dic)

In [6]:
df = pd.DataFrame(country_list)

In [7]:
df.head()

,OBJECTID,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
0,1,Alberta,Canada,1.585887e+12,53.9333,-116.5765,969,0,13
1,2,Anguilla,United Kingdom,1.585887e+12,18.2206,-63.0686,3,0,0
2,3,Anhui,China,1.583645e+12,31.8257,117.2264,990,984,6
3,4,Aruba,Netherlands,1.585887e+12,12.5211,-69.9683,60,1,0
4,5,Australian Capital Territory,Australia,1.585887e+12,-35.4735,149.0124,87,11,1


In [8]:
df.Last_Update = pd.to_datetime(
    df.Last_Update/1000,unit='s').dt.tz_localize('utc').dt.tz_convert('Asia/Tokyo').dt.strftime('%Y-%m-%d %H:%M')

In [9]:
df.head()

,OBJECTID,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
0,1,Alberta,Canada,2020-04-03 13:12,53.9333,-116.5765,969,0,13
1,2,Anguilla,United Kingdom,2020-04-03 13:05,18.2206,-63.0686,3,0,0
2,3,Anhui,China,2020-03-08 14:19,31.8257,117.2264,990,984,6
3,4,Aruba,Netherlands,2020-04-03 13:05,12.5211,-69.9683,60,1,0
4,5,Australian Capital Territory,Australia,2020-04-03 13:12,-35.4735,149.0124,87,11,1


In [10]:
df.loc[df[df['Province_State'] == 'Hong Kong'].index,'Country_Region'] = 'Hong Kong'

In [11]:
df.loc[df[df['Province_State'] == 'Macau'].index,'Country_Region'] = 'Macau'

In [12]:
df.head()

,OBJECTID,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
0,1,Alberta,Canada,2020-04-03 13:12,53.9333,-116.5765,969,0,13
1,2,Anguilla,United Kingdom,2020-04-03 13:05,18.2206,-63.0686,3,0,0
2,3,Anhui,China,2020-03-08 14:19,31.8257,117.2264,990,984,6
3,4,Aruba,Netherlands,2020-04-03 13:05,12.5211,-69.9683,60,1,0
4,5,Australian Capital Territory,Australia,2020-04-03 13:12,-35.4735,149.0124,87,11,1


In [13]:
latest = df.drop(['OBJECTID','Last_Update','Lat','Long_'],axis=1).groupby('Country_Region').sum()

In [14]:
latest['Active'] = latest.Confirmed - latest.Deaths - latest.Recovered

In [15]:
latest.head()

,Confirmed,Recovered,Deaths,Active
Country_Region,,,,
Afghanistan,273,10,6,257
Albania,277,76,16,185
Algeria,986,61,86,839
Andorra,428,10,15,403
Angola,8,1,2,5


In [16]:
latest.to_csv('latest/latest_global_'+datetime.now().strftime('%Y%m%d_%I%M%p')+'.csv',index=False,encoding='utf-8-sig')

In [17]:
confirmed_df = pd.read_csv('data/confirmed_global_'+datetime.now().strftime('%Y%m%d')+'.csv',index_col='Country/Region')

In [18]:
confirmed_df.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02
Country/Region,,,,,,,,,,,,,,,,,,,,,
US,1,1,2,2,5,5,5,5,5,7,...,53740,65778,83836,101657,121478,140886,161807,188172,213372,243453
Italy,0,0,0,0,0,0,0,0,0,2,...,69176,74386,80589,86498,92472,97689,101739,105792,110574,115242
Spain,0,0,0,0,0,0,0,0,0,0,...,39885,49515,57786,65719,73235,80110,87956,95923,104118,112065
Germany,0,0,0,0,0,1,4,4,4,5,...,32986,37323,43938,50871,57695,62095,66885,71808,77872,84794
China,547,639,916,1399,2062,2863,5494,6070,8124,9783,...,81180,81221,81298,81345,81401,81444,81478,81524,81555,81589


In [19]:
confirmed_df[datetime.now().strftime('%Y-%m-%d')] = confirmed_df.index.map(latest.Confirmed.to_dict())

In [20]:
confirmed_df.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03
Country/Region,,,,,,,,,,,,,,,,,,,,,
US,1,1,2,2,5,5,5,5,5,7,...,65778,83836,101657,121478,140886,161807,188172,213372,243453,245540
Italy,0,0,0,0,0,0,0,0,0,2,...,74386,80589,86498,92472,97689,101739,105792,110574,115242,115242
Spain,0,0,0,0,0,0,0,0,0,0,...,49515,57786,65719,73235,80110,87956,95923,104118,112065,112065
Germany,0,0,0,0,0,1,4,4,4,5,...,37323,43938,50871,57695,62095,66885,71808,77872,84794,84794
China,547,639,916,1399,2062,2863,5494,6070,8124,9783,...,81221,81298,81345,81401,81444,81478,81524,81555,81589,81613


In [21]:
confirmed_df.sort_values(confirmed_df.columns[-1],ascending=False).to_csv(
    'latest/latest_confirmed_'+datetime.today().strftime('%Y%m%d_%I%M%p')+'.csv',encoding='utf-8-sig')

In [22]:
new_cases = confirmed_df.diff(axis=1)

In [23]:
new_cases.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03
Country/Region,,,,,,,,,,,,,,,,,,,,,
US,NaN,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,...,12038.0,18058.0,17821.0,19821.0,19408.0,20921.0,26365.0,25200.0,30081.0,2087.0
Italy,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,5210.0,6203.0,5909.0,5974.0,5217.0,4050.0,4053.0,4782.0,4668.0,0.0
Spain,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9630.0,8271.0,7933.0,7516.0,6875.0,7846.0,7967.0,8195.0,7947.0,0.0
Germany,NaN,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,...,4337.0,6615.0,6933.0,6824.0,4400.0,4790.0,4923.0,6064.0,6922.0,0.0
China,NaN,92.0,277.0,483.0,663.0,801.0,2631.0,576.0,2054.0,1659.0,...,41.0,77.0,47.0,56.0,43.0,34.0,46.0,31.0,34.0,24.0


In [24]:
new_cases.sort_values(new_cases.columns[-1],ascending=False).to_csv(
    'latest/latest_new_cases_'+datetime.today().strftime('%Y%m%d_%I%M%p')+'.csv',encoding='utf-8-sig')

In [25]:
deaths_df = pd.read_csv('data/deaths_global_'+datetime.now().strftime('%Y%m%d')+'.csv',index_col='Country/Region')

In [26]:
deaths_df.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02
Country/Region,,,,,,,,,,,,,,,,,,,,,
Italy,0,0,0,0,0,0,0,0,0,0,...,6820,7503,8215,9134,10023,10779,11591,12428,13155,13915
Spain,0,0,0,0,0,0,0,0,0,0,...,2808,3647,4365,5138,5982,6803,7716,8464,9387,10348
US,0,0,0,0,0,0,0,0,0,0,...,706,942,1209,1581,2026,2467,2978,3873,4757,5926
France,0,0,0,0,0,0,0,0,0,0,...,1102,1333,1698,1997,2317,2611,3030,3532,4043,5398
China,17,18,26,42,56,82,131,133,171,213,...,3277,3281,3287,3292,3295,3300,3304,3305,3312,3318


In [27]:
deaths_df[datetime.now().strftime('%Y-%m-%d')] = deaths_df.index.map(latest.Deaths.to_dict())

In [28]:
deaths_df.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03
Country/Region,,,,,,,,,,,,,,,,,,,,,
Italy,0,0,0,0,0,0,0,0,0,0,...,7503,8215,9134,10023,10779,11591,12428,13155,13915,13915
Spain,0,0,0,0,0,0,0,0,0,0,...,3647,4365,5138,5982,6803,7716,8464,9387,10348,10348
US,0,0,0,0,0,0,0,0,0,0,...,942,1209,1581,2026,2467,2978,3873,4757,5926,6053
France,0,0,0,0,0,0,0,0,0,0,...,1333,1698,1997,2317,2611,3030,3532,4043,5398,5398
China,17,18,26,42,56,82,131,133,171,213,...,3281,3287,3292,3295,3300,3304,3305,3312,3318,3322


In [29]:
deaths_df.sort_values(deaths_df.columns[-1],ascending=False).to_csv(
    'latest/latest_deaths_'+datetime.today().strftime('%Y%m%d_%I%M%p')+'.csv',encoding='utf-8-sig')

In [30]:
new_deaths = deaths_df.diff(axis=1)

In [31]:
new_deaths.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03
Country/Region,,,,,,,,,,,,,,,,,,,,,
Italy,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,683.0,712.0,919.0,889.0,756.0,812.0,837.0,727.0,760.0,0.0
Spain,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,839.0,718.0,773.0,844.0,821.0,913.0,748.0,923.0,961.0,0.0
US,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,236.0,267.0,372.0,445.0,441.0,511.0,895.0,884.0,1169.0,127.0
France,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,231.0,365.0,299.0,320.0,294.0,419.0,502.0,511.0,1355.0,0.0
China,NaN,1.0,8.0,16.0,14.0,26.0,49.0,2.0,38.0,42.0,...,4.0,6.0,5.0,3.0,5.0,4.0,1.0,7.0,6.0,4.0


In [32]:
new_deaths.sort_values(new_deaths.columns[-1],ascending=False).to_csv(
    'latest/latest_new_deaths_'+datetime.today().strftime('%Y%m%d_%I%M%p')+'.csv',encoding='utf-8-sig')

In [33]:
recovered_df = pd.read_csv('data/recovered_global_'+datetime.now().strftime('%Y%m%d')+'.csv',index_col='Country/Region')

In [34]:
recovered_df.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02
Country/Region,,,,,,,,,,,,,,,,,,,,,
China,28,30,36,39,49,58,101,120,135,214,...,73169,73661,74061,74600,74978,75460,75790,76068,76248,76408
Spain,0,0,0,0,0,0,0,0,0,0,...,3794,5367,7015,9357,12285,14709,16780,19259,22647,26743
Germany,0,0,0,0,0,0,0,0,0,0,...,3243,3547,5673,6658,8481,9211,13500,16100,18700,22440
Italy,0,0,0,0,0,0,0,0,0,0,...,8326,9362,10361,10950,12384,13030,14620,15729,16847,18278
Iran,0,0,0,0,0,0,0,0,0,0,...,8913,9625,10457,11133,11679,12391,13911,14656,15473,16711


In [35]:
recovered_df[datetime.now().strftime('%Y-%m-%d')] = recovered_df.index.map(latest.Recovered.to_dict())

In [36]:
recovered_df.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03
Country/Region,,,,,,,,,,,,,,,,,,,,,
China,28,30,36,39,49,58,101,120,135,214,...,73661,74061,74600,74978,75460,75790,76068,76248,76408,76567
Spain,0,0,0,0,0,0,0,0,0,0,...,5367,7015,9357,12285,14709,16780,19259,22647,26743,26743
Germany,0,0,0,0,0,0,0,0,0,0,...,3547,5673,6658,8481,9211,13500,16100,18700,22440,22440
Italy,0,0,0,0,0,0,0,0,0,0,...,9362,10361,10950,12384,13030,14620,15729,16847,18278,18278
Iran,0,0,0,0,0,0,0,0,0,0,...,9625,10457,11133,11679,12391,13911,14656,15473,16711,16711


In [37]:
recovered_df.sort_values(recovered_df.columns[-1],ascending=False).to_csv(
    'latest/latest_recovered_'+datetime.today().strftime('%Y%m%d_%I%M%p')+'.csv',encoding='utf-8-sig')

In [38]:
new_recovered = recovered_df.diff(axis=1)

In [39]:
new_recovered.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03
Country/Region,,,,,,,,,,,,,,,,,,,,,
China,NaN,2.0,6.0,3.0,10.0,9.0,43.0,19.0,15.0,79.0,...,492.0,400.0,539.0,378.0,482.0,330.0,278.0,180.0,160.0,159.0
Spain,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1573.0,1648.0,2342.0,2928.0,2424.0,2071.0,2479.0,3388.0,4096.0,0.0
Germany,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,304.0,2126.0,985.0,1823.0,730.0,4289.0,2600.0,2600.0,3740.0,0.0
Italy,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1036.0,999.0,589.0,1434.0,646.0,1590.0,1109.0,1118.0,1431.0,0.0
Iran,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,712.0,832.0,676.0,546.0,712.0,1520.0,745.0,817.0,1238.0,0.0


In [40]:
new_recovered.sort_values(new_recovered.columns[-1],ascending=False).to_csv(
    'latest/latest_new_recovered_'+datetime.today().strftime('%Y%m%d_%I%M%p')+'.csv',encoding='utf-8-sig')

In [41]:
active_df = pd.read_csv('data/active_global_'+datetime.now().strftime('%Y%m%d')+'.csv',index_col='Country/Region')

In [42]:
active_df.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02
Country/Region,,,,,,,,,,,,,,,,,,,,,
US,1,1,2,2,5,5,5,5,5,7,...,52686,64475,81946,99207,118380,135754,153185,177275,200141,228526
Italy,0,0,0,0,0,0,0,0,0,2,...,54030,57521,62013,66414,70065,73880,75528,77635,80572,83049
Spain,0,0,0,0,0,0,0,0,0,0,...,33283,40501,46406,51224,54968,58598,63460,68200,72084,74974
Germany,0,0,0,0,0,1,4,4,4,5,...,29586,33570,37998,43871,48781,52351,52740,54933,58252,61247
France,0,0,2,3,3,3,4,5,5,5,...,18270,20360,22898,25698,30064,30871,34176,39782,42653,41983


In [43]:
active_df[datetime.now().strftime('%Y-%m-%d')] = active_df.index.map(latest.Active.to_dict())

In [44]:
active_df.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03
Country/Region,,,,,,,,,,,,,,,,,,,,,
US,1,1,2,2,5,5,5,5,5,7,...,64475,81946,99207,118380,135754,153185,177275,200141,228526,230339
Italy,0,0,0,0,0,0,0,0,0,2,...,57521,62013,66414,70065,73880,75528,77635,80572,83049,83049
Spain,0,0,0,0,0,0,0,0,0,0,...,40501,46406,51224,54968,58598,63460,68200,72084,74974,74974
Germany,0,0,0,0,0,1,4,4,4,5,...,33570,37998,43871,48781,52351,52740,54933,58252,61247,61247
France,0,0,2,3,3,3,4,5,5,5,...,20360,22898,25698,30064,30871,34176,39782,42653,41983,41983


In [45]:
active_df.sort_values(active_df.columns[-1],ascending=False).to_csv(
    'latest/latest_active_'+datetime.today().strftime('%Y%m%d_%I%M%p')+'.csv',encoding='utf-8-sig')

In [46]:
active_change = active_df.diff(axis=1)

In [47]:
active_change.head()

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03
Country/Region,,,,,,,,,,,,,,,,,,,,,
US,NaN,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,...,11789.0,17471.0,17261.0,19173.0,17374.0,17431.0,24090.0,22866.0,28385.0,1813.0
Italy,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,3491.0,4492.0,4401.0,3651.0,3815.0,1648.0,2107.0,2937.0,2477.0,0.0
Spain,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7218.0,5905.0,4818.0,3744.0,3630.0,4862.0,4740.0,3884.0,2890.0,0.0
Germany,NaN,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,...,3984.0,4428.0,5873.0,4910.0,3570.0,389.0,2193.0,3319.0,2995.0,0.0
France,NaN,0.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,2090.0,2538.0,2800.0,4366.0,807.0,3305.0,5606.0,2871.0,-670.0,0.0


In [48]:
active_change.sort_values(active_change.columns[-1],ascending=False).to_csv(
    'latest/latest_active_change_'+datetime.today().strftime('%Y%m%d_%I%M%p')+'.csv',encoding='utf-8-sig')

In [49]:
countries = pd.read_csv('countries.csv')

In [50]:
countries.head()

,Country,Japanese,Code,Region
0,Afghanistan,アフガニスタン,AF,Asia
1,Albania,アルバニア,AL,Europe
2,Algeria,アルジェリア,DZ,Africa
3,Andorra,アンドラ,AD,Europe
4,Angola,アンゴラ,AO,Africa


In [51]:
latest.index = latest.index.map(countries.set_index('Country').Japanese.to_dict())

In [52]:
latest.head()

,Confirmed,Recovered,Deaths,Active
Country_Region,,,,
アフガニスタン,273,10,6,257
アルバニア,277,76,16,185
アルジェリア,986,61,86,839
アンドラ,428,10,15,403
アンゴラ,8,1,2,5


In [53]:
latest.columns = ['累積感染者数','累積回復者数','累積死者数','現行感染者数']

In [54]:
latest.head()

,累積感染者数,累積回復者数,累積死者数,現行感染者数
Country_Region,,,,
アフガニスタン,273,10,6,257
アルバニア,277,76,16,185
アルジェリア,986,61,86,839
アンドラ,428,10,15,403
アンゴラ,8,1,2,5


In [55]:
latest.sum().rename(datetime.today().strftime('%Y-%m-%d'))

累積感染者数    1016128
累積回復者数     211615
累積死者数       53146
現行感染者数     751367
Name: 2020-04-03, dtype: int64

In [56]:
total = pd.read_excel('covid19_'+datetime.today().strftime('%Y%m%d')+'.xlsx',sheet_name='一覧',index_col=0).append(
    latest.sum().rename(datetime.today().strftime('%Y-%m-%d')),sort=True).sort_index(ascending=False)

In [57]:
total.head()

,新規回復者数,新規感染者数,新規死者数,現行感染者増減数,現行感染者数,累積回復者数,累積感染者数,累積死者数
2020-04-03,NaN,NaN,NaN,NaN,751367.0,211615.0,1016128.0,53146.0
2020-04-02,17086.0,80552.0,6174.0,57292.0,749911.0,210263.0,1013157.0,52983.0
2020-04-01,15143.0,75118.0,4702.0,55273.0,692619.0,193177.0,932605.0,46809.0
2020-03-31,13468.0,75122.0,4525.0,57129.0,637346.0,178034.0,857487.0,42107.0
2020-03-30,15484.0,62248.0,3657.0,43107.0,580217.0,164566.0,782365.0,37582.0


In [58]:
total.iloc[0,0] = total.iloc[0,5] - total.iloc[1,5]

In [59]:
total.iloc[0,1] = total.iloc[0,6] - total.iloc[1,6]

In [60]:
total.iloc[0,2] = total.iloc[0,7] - total.iloc[1,7]

In [61]:
total.iloc[0,3] = total.iloc[0,4] - total.iloc[1,4]

In [62]:
total = total.astype(int)

In [63]:
total.head()

,新規回復者数,新規感染者数,新規死者数,現行感染者増減数,現行感染者数,累積回復者数,累積感染者数,累積死者数
2020-04-03,1352,2971,163,1456,751367,211615,1016128,53146
2020-04-02,17086,80552,6174,57292,749911,210263,1013157,52983
2020-04-01,15143,75118,4702,55273,692619,193177,932605,46809
2020-03-31,13468,75122,4525,57129,637346,178034,857487,42107
2020-03-30,15484,62248,3657,43107,580217,164566,782365,37582


In [64]:
changes = pd.concat([new_cases[new_cases.columns[-1]],
                     new_deaths[new_deaths.columns[-1]],
                     new_recovered[new_recovered.columns[-1]],
                     active_change[active_change.columns[-1]]],axis=1,sort=True)

In [65]:
changes.columns = ['new cases','new deaths','new recovered','active change']

In [66]:
changes.head()

,new cases,new deaths,new recovered,active change
Afghanistan,0.0,0.0,0.0,0.0
Albania,0.0,0.0,0.0,0.0
Algeria,0.0,0.0,0.0,0.0
Andorra,0.0,0.0,0.0,0.0
Angola,0.0,0.0,0.0,0.0


In [67]:
changes.index = changes.index.map(countries.set_index('Country').Japanese.to_dict())

In [68]:
changes.columns = ['新規感染者数','新規死者数','新規回復者数','現行感染者増減数']

In [69]:
changes = changes.astype(int)

In [70]:
latest = latest.join(changes)

In [71]:
latest.head()

,累積感染者数,累積回復者数,累積死者数,現行感染者数,新規感染者数,新規死者数,新規回復者数,現行感染者増減数
Country_Region,,,,,,,,
アフガニスタン,273,10,6,257,0,0,0,0
アルバニア,277,76,16,185,0,0,0,0
アルジェリア,986,61,86,839,0,0,0,0
アンドラ,428,10,15,403,0,0,0,0
アンゴラ,8,1,2,5,0,0,0,0


In [72]:
with pd.ExcelWriter('covid19_latest_'+datetime.now().strftime('%Y%m%d_%I%M%p')+'.xlsx') as writer:  
    latest.sort_values('新規感染者数',ascending=False).to_excel(writer,sheet_name='最新')
    total.to_excel(writer,sheet_name='合計')

* Lancet Inf Dis Article: https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30120-1/fulltext.
* Mobile Version: https://arcgis.com/apps/opsdashboard/index.html#/85320e2ea5424dfaaa75ae62e5c06e61.
* Visualization: JHU CSSE https://systems.jhu.edu/.
* Automation Support: Esri Living Atlas team https://livingatlas.arcgis.com/en/ and JHU APL https://www.jhuapl.edu/.
* Contact US: jhusystems@gmail.com.
* FAQ: https://systems.jhu.edu/research/public-health/2019-ncov-map-faqs/.

* Data sources:
** WHO https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports,
** CDC https://www.cdc.gov/coronavirus/2019-ncov/index.html,
** ECDC https://www.ecdc.europa.eu/en/geographical-distribution-2019-ncov-cases,
** NHC http://www.nhc.gov.cn/xcs/yqtb/list_gzbd.shtml,
** DXY https://ncov.dxy.cn/ncovh5/view/en_pneumonia?from=dxy&source=&link=&share=,
** 1point3acres https://coronavirus.1point3acres.com/,
** Worldometers.info https://www.worldometers.info/coronavirus/,
** BNO https://bnonews.com/index.php/2020/04/the-latest-coronavirus-cases/,
** state and national government health departments, and local media reports.
* Read more in this blog https://systems.jhu.edu/research/public-health/ncov/. 

* Downloadable database: GitHub: https://github.com/CSSEGISandData/COVID-19.
* Feature layer: https://www.arcgis.com/home/item.html?id=c0b356e20b30490c8b8b4c7bb9554e7c.

* Confirmed cases include presumptive positive cases.

* Recovered cases outside China are estimates based on local media reports, and may be substantially lower than the true number.

* Active cases = total confirmed - total recovered - total deaths.

* Point level: County level - US; Province/State level - China, Canada, Australia; Country level - other countries. All points (except for Australia) shown on the map are based on geographic centroids, and are not representative of a specific address, building or any location at a spatial scale finer than a province/state. Australian dots are located at the centroid of the largest city in each state.

* Time Zones: lower-left corner indicator - your local time; lower-right corner plot - UTC.

* Note: All cases of COVID-19 in repatriated US citizens from the Diamond Princess are grouped together. These individuals have been assigned to various quarantine locations (in military bases and hospitals) around the US. This grouping is consistent with the CDC.

* The names of locations included on the Website correspond with the official designations used by the U.S. Department of State. 

* Terms and Conditions of Website Use:  This website and its contents herein, including all data, mapping, and analysis (“Website”), copyright 2020 Johns Hopkins University, all rights reserved, is provided to the public strictly for public health, educational, and academic research purposes. Redistribution of the Website or the aggregated data set underlying the Website is strictly prohibited.  You are welcome to link to the Website, however.  The Website relies upon publicly available data from multiple sources that do not always agree. The Johns Hopkins University hereby disclaims any and all representations and warranties with respect to the Website, including accuracy, fitness for use, reliability, and non-infringement. Reliance on the Website for medical guidance or use of the Website in commerce is strictly prohibited.  Any use of the Johns Hopkins’ names, logos, trademarks, and trade dress for promotional or commercial purposes is strictly prohibited.

* Visit the Johns Hopkins Coronavirus Resource Center https://coronavirus.jhu.edu/ where our experts help to advance understanding of the virus, inform the public, and brief policymakers in order to guide a response, improve care, and save lives.